# Визуализация LiDAR точек

Этот ноутбук показывает облако точек с раскраской по классам:
- если есть `Predicted_Classification`, используется предсказание
- иначе используется `Classification`


In [ ]:
import pandas as pd
import numpy as np
import os

try:
    import open3d as o3d
except Exception as e:
    print("Open3D не установлен. Установите: pip install open3d")
    raise


In [ ]:
# Выберите файл данных и (опционально) файл предсказаний
# Примеры:
# data_path = "Files/Mar16/LiDAR/Mar16_test.txt"
# data_path = "Files/Mar18/LiDAR/Mar18_test.txt"

data_path = "Files/Mar18/LiDAR/Mar18_test.txt"

# Если есть файл с предсказаниями модели (Predicted_Classification)
predictions_path = "predictions_pointnetpp.txt"  # или None


In [ ]:
df = pd.read_csv(data_path, sep="\t")
print(df.head())

# Если есть предсказания, используем их
labels = None
if predictions_path and os.path.exists(predictions_path):
    pred_df = pd.read_csv(predictions_path, sep="\t")
    if "Predicted_Classification" in pred_df.columns:
        labels = pred_df["Predicted_Classification"].to_numpy(dtype=np.int32)
        print("Используются предсказания")

# Иначе используем ground truth
if labels is None and "Classification" in df.columns:
    labels = df["Classification"].to_numpy(dtype=np.int32)
    print("Используется ground truth")

if labels is None:
    raise ValueError("Нет меток для визуализации. Укажите predictions_path или файл с Classification")

xyz = df[["X", "Y", "Z"]].to_numpy(dtype=np.float64)


In [ ]:
# Дискретная палитра для классов
num_classes = int(labels.max()) + 1
palette = np.random.RandomState(42).rand(num_classes, 3)  # стабильные цвета
colors = palette[labels]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Центрирование облака
pcd.translate(-pcd.get_center())

In [ ]:
o3d.visualization.draw_geometries([pcd], width=900, height=700)